<a href="https://colab.research.google.com/github/omerfarukpala/miuul/blob/main/crm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RFM ile Müşteri Segmentasyonu (Customer Segmentation with RFM)



#1. İs Problemi (Business Problem)
#2. Veriyi Anlama (Data Understanding)
#3. Veri Hazırlama (Data Preparation)
#4. RFM Metriklerinin Hesaplanması (Calculating RFM Metrics)
#5. RFM Skorlarının Hesaplanması (Calculating RFM Scores)
# 6. RFM Segmentlerinin Oluşturulması ve Analiz Edilmesi (Creating & Analysing RFM Segments)
#7. Tüm Sürecin Fonksiyonlaştırılması

Is Problemi (Business Problem)

Bir e-ticaret şirketi müşterilerini segmentlere ayırıp bu segmentlere göre

pazarlama stratejileri belirlemek istiyor.

Veri Seti Hikayesi

 https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

 Online Retail II isimli veri seti İngiltere
merkezli online bir satış mağazasının # 01/12/2009 09/12/2011 tarihleri arasındaki satışlarını içeriyor.



* InvoiceNo: Invoice Number(fatura nuarası) her işleme yani faturaya eşsiz numara
* StockCode: Product code (ürün kodu) eşsiz
* Description: Product name (ürün ismi)
* Quantity: Number of products (ürün adedi) faturadaki ürünlerden kaçar tane satıldığını ifade eder.
* InvoiceDate: Invoice date Fatura tarihi ve zaman
* UnitPrice: Invoice price ( Sterling ) ürün fiyatı
* CustomerID: Unique customer number eşsiz müşteri numarası
* Country: Country name müşterinin yaşadığı ülke



# **Importing the libraries**

In [164]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# **Importing the dataset**

In [167]:
df_ = pd.read_excel("/content/sample_data/online_retail_II.xlsx", sheet_name="Year 2009-2010")

In [168]:
df = df_.copy()

In [169]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom


In [170]:
df.shape

(525461, 8)

In [171]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [172]:
# essiz urun sayisi nedir?
df["Description"].nunique()

4681

In [173]:
df["Description"].value_counts().head()

WHITE HANGING HEART T-LIGHT HOLDER    3549
REGENCY CAKESTAND 3 TIER              2212
STRAWBERRY CERAMIC TRINKET BOX        1843
PACK OF 72 RETRO SPOT CAKE CASES      1466
ASSORTED COLOUR BIRD ORNAMENT         1457
Name: Description, dtype: int64

her bir eşsiz description için quantitiy sayısını hesaplayıp topluyor

In [174]:
df.groupby("Description").agg({"Quantity": "sum"}).head()

,Quantity
Description,
21494,-720
22467,-2
22719,2
DOORMAT UNION JACK GUNS AND ROSES,179
3 STRIPEY MICE FELTCRAFT,690


quantity sütununa göre azalan şekilde sıralama

In [175]:
df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False).head()

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,57733
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54698
BROCADE RING PURSE,47647
PACK OF 72 RETRO SPOT CAKE CASES,46106
ASSORTED COLOUR BIRD ORNAMENT,44925


invoice için benzersiz değişken sayısı

In [176]:
df["Invoice"].nunique()

28816

"TotalPrice" adında yeni bir sürun oluşturduk.
bu sütunda toplam kazancı saklayacağız

In [177]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

faturalarda toplam kazanç

In [178]:
df.groupby("Invoice").agg({"TotalPrice": "sum"}).head()

,TotalPrice
Invoice,
489434,505.300
489435,145.800
489436,630.330
489437,310.750
489438,2286.240


# 3. Veri Hazırlama (Data Preparation)

In [179]:
df.shape

(525461, 9)

In [180]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
TotalPrice          0
dtype: int64

quantity de 75 de 12.0 iken max da 19152 gibi bir değer var

In [181]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,525461.000,10.338,107.424,-9600.000,1.000,3.000,10.000,19152.000
Price,525461.000,4.689,146.127,-53594.360,1.250,2.100,4.210,25111.090
Customer ID,417534.000,15360.645,1680.811,12346.000,13983.000,15311.000,16799.000,18287.000
TotalPrice,525461.000,18.155,160.333,-53594.360,3.750,9.950,17.700,25111.090


iadelerden kaynaklı (-) değerleri siliyoruz

In [182]:
 df = df[(df['Quantity'] > 0)]

nan bulunanları siliyoruz

descriptionda 2928 tane vardı

In [183]:
 df.dropna(inplace=True)

In [184]:
# df = df[df["Invoice"].str.contains("C", na=False)]

# 4. RFM Metriklerinin Hesaplanması (Calculating RFM Metrics)

In [213]:
# Recency, Frequency, Monetary
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom,83.400
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom,81.000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom,81.000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom,100.800
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom,30.000


In [214]:
df["InvoiceDate"].max()

Timestamp('2010-12-09 20:01:00')

In [215]:
today_date = dt.datetime(2010, 12, 11)
type(today_date)

datetime.datetime

bir sözlük oluşturduk.

Recency, Frequency, Monetary oluşturduk.

son işlemden bu zamana geçen süre

kaç tane işlem yapılmış

bu kişi bize toplam ne kadar kazandırmış

In [216]:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                     'Invoice': lambda Invoice: Invoice.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.000,165,11,372.860
12347.000,3,2,1323.320
12348.000,74,1,222.160
12349.000,43,3,2671.140
12351.000,11,1,300.930


In [217]:
rfm.columns = ['recency', 'frequency', 'monetary']

In [218]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4314.000,91.270,96.944,1.000,18.000,53.000,136.000,374.000
frequency,4314.000,4.454,8.169,1.000,1.000,2.000,5.000,205.000
monetary,4314.000,2047.289,8912.523,0.000,307.950,705.550,1722.802,349164.350


In [219]:
rfm = rfm[rfm["monetary"] > 0]
rfm.shape

(4312, 3)

# 5. RFM Skorlarının Hesaplanması (Calculating RFM Scores)

çeyrek değerlere göre bölme işlemi yapıyoruz.

mesela 5 parçaya böleceğiz

ve 1-5 arası yeni değerler vereceğiz.

on işlem tarihi bizim için küçük olması daha iyi o sebeple bu şekilde yaptık.

In [220]:
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])

0-100, 0-20, 20-40, 40-60, 60-80, 80-100

aynı araliğa çok fazla değer gelme hatasını çözmek için rank methodunu kullanıyoruz.

bu şekilde ilk gördüğünü 1 e atadı.

In [221]:
rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

In [222]:
rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

In [223]:
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str))

In [224]:
rfm

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12346.000,165,11,372.860,2,5,2,25
12347.000,3,2,1323.320,5,2,4,52
12348.000,74,1,222.160,2,1,1,21
12349.000,43,3,2671.140,3,3,5,33
12351.000,11,1,300.930,5,1,2,51
...,...,...,...,...,...,...,...
18283.000,18,6,641.770,4,5,3,45
18284.000,67,1,461.680,3,2,2,32
18285.000,296,1,427.000,1,2,2,12


In [225]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4312.000,91.173,96.861,1.000,18.000,53.000,136.000,374.000
frequency,4312.000,4.456,8.170,1.000,1.000,2.000,5.000,205.000
monetary,4312.000,2048.238,8914.481,2.950,307.988,706.020,1723.142,349164.350


şampiyon müşterileri görmek istiyoruz.

In [226]:
rfm[rfm["RFM_SCORE"] == "55"]

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12415.000,11,7,19543.840,5,5,5,55
12431.000,9,13,4370.520,5,5,5,55
12471.000,10,49,20139.740,5,5,5,55
12472.000,5,13,11308.480,5,5,5,55
12474.000,14,13,5048.660,5,5,5,55
...,...,...,...,...,...,...,...
18225.000,1,15,7545.140,5,5,5,55
18226.000,14,15,6650.830,5,5,5,55
18229.000,2,10,3526.810,5,5,5,55


In [227]:

rfm[rfm["RFM_SCORE"] == "11"]

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12355.000,203,1,488.210,1,1,2,11
12362.000,374,1,130.000,1,1,1,11
12366.000,269,1,500.240,1,1,2,11
12368.000,264,1,917.700,1,1,3,11
12378.000,198,1,1407.700,1,1,4,11
...,...,...,...,...,...,...,...
15928.000,292,1,293.530,1,1,2,11
15929.000,280,1,594.000,1,1,3,11
15941.000,273,1,405.000,1,1,2,11


# 6. RFM Segmentlerinin Oluşturulması ve Analiz Edilmesi (Creating & Analysing RFM Segments)

regex

r sinde 5 f sinde 5 görürsen shampion yaz

RFM isimlendirmesi

In [228]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',             # riskli durumda
    r'[1-2]5': 'cant_loose',              # kaybetme
    r'3[1-2]': 'about_to_sleep',          # uyku halinde
    r'33': 'need_attention',              #
    r'[3-4][4-5]': 'loyal_customers',     # sadık müşteri
    r'41': 'promising',     #
    r'51': 'new_customers', # yeni müşteri
    r'[4-5][2-3]': 'potential_loyalists', # bizim için sadık bir müşteri olabilir.
    r'5[4-5]': 'champions' # şampiyonlar
}

In [229]:
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)

In [230]:
rfm = rfm[["recency", "frequency", "monetary", "segment"]]

In [231]:
rfm.index = rfm.index.astype(int)

In [232]:
rfm

,recency,frequency,monetary,segment
Customer ID,,,,
12346,165,11,372.860,cant_loose
12347,3,2,1323.320,potential_loyalists
12348,74,1,222.160,hibernating
12349,43,3,2671.140,need_attention
12351,11,1,300.930,new_customers
...,...,...,...,...
18283,18,6,641.770,loyal_customers
18284,67,1,461.680,about_to_sleep
18285,296,1,427.000,hibernating


segmentlerine göre bir gruplama yaptık

ortalamalarını ve toplam sayılarını gösterdik



In [233]:
rfm = rfm[["recency", "frequency", "monetary", "segment"]].groupby("segment").agg(["mean","count"])

In [234]:
rfm

recency       frequency       monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep       53.819   343     1.201   343  441.320   343
at_Risk             152.159   611     3.074   611 1188.878   611
cant_loose          124.117    77     9.117    77 4099.450    77
champions             7.119   663    12.554   663 6852.264   663
hibernating         213.886  1015     1.126  1015  403.978  1015
loyal_customers      36.287   742     6.830   742 2746.067   742
need_attention       53.266   207     2.449   207 1060.357   207
new_customers         8.580    50     1.000    50  386.199    50
potential_loyalists  18.793   517     2.017   517  729.511   517
promising            25.747    87     1.000    87  367.087    87

In [246]:
# saving a separate file the last dataframe
   # if csv:
    #    rfm.to_csv('rfm.csv')

    #return rfm

# call the dataset again
#df = df_.copy()
#df.columns = [col.lower() for col in df.columns]
#rfm_new = create_rfm(df)     # can not create the csv file
#rfm_new = create_rfm(df, csv=True) # create the csv file
#rfm_new.head()